# Load Dependencies and Libraries

In [1]:
#Dependencies
import numpy as np
import pandas as pd
import re
import time

from requests import get
from bs4 import BeautifulSoup
from time import sleep
from random import randint
from warnings import warn
from IPython.core.display import clear_output

# Set Source URL

In [2]:
#Set the URL source and pull the first page of listings on Craigslist for Los Angeles
url = "https://losangeles.craigslist.org/d/housing/search/hhh"
response = get('https://losangeles.craigslist.org/d/housing/search/hhh') 

# Review code for information you want to extract

In [3]:
#grab the HTML as a BS4 soup object
soup = BeautifulSoup(response.text, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width,initial-scale=1" name="viewport"/>
  <meta content="craigslist" property="og:site_name"/>
  <meta content="preview" name="twitter:card"/>
  <meta content="los angeles housing - craigslist" property="og:title"/>
  <meta content="los angeles housing - craigslist" name="description"/>
  <meta content="los angeles housing - craigslist" property="og:description"/>
  <meta content="https://losangeles.craigslist.org/d/housing/search/hhh" property="og:url"/>
  <title>
   los angeles housing - craigslist
  </title>
  <link href="https://losangeles.craigslist.org/d/housing/search/hhh" rel="canonical"/>
  <link href="https://losangeles.craigslist.org/d/housing/search/hhh?s=120" rel="next"/>
  <script id="ld_breadcrumb_data" type="application/ld+json">
   {"@context":"https://schema.org","itemListElement":[{"item":{"name":"losangeles.craigslist.org",

In [4]:
#Identify the containers with the necessary information
listings = soup.find_all('li', class_= 'result-row')

In [5]:
#Isolate the first listing
listing1 = listings[0]

#Verify listing isolation is correct
print(listing1.text[:500])



$2,485



favorite this post

Oct 10

1br - We-Work Space, 29 Seat Screening Room, Courtesy Patrol


$2,485

                    1br -
                    700ft2 -
                
 (4500 Woodman Ave, Sherman Oaks, CA san fernando valley )

pic


hide this posting



restore
restore this posting






# Extract data in one listing to prepare for looping

In [6]:
#Identify the price of the first listing, by isolating the text contained in the <a> tag
listing1_price = listing1.a.text

#Remove the newline code by using the strip() function
int(listing1_price.strip().replace("$", "").replace(",", ""))

2485

In [7]:
#Isolate the time of the listing in datetime format
listing1_time = listing1.find('time', class_= 'result-date')
listing1_datetime = listing1_time['datetime']
listing1_datetime 

'2021-10-10 09:18'

In [8]:
#Isolate the title and link of the listing in the <a> and 'result-title' class
listing1_title = listing1.find('a', class_='result-title')
listing1_titletext = listing1_title.text
listing1_titletext

'1br - We-Work Space, 29 Seat Screening Room, Courtesy Patrol'

In [9]:
#Exrtact the link by pulling the href variable
listing1_link = listing1_title['href']
listing1_link

'https://losangeles.craigslist.org/sfv/apa/d/sherman-oaks-1br-we-work-space-29-seat/7385551952.html'

In [10]:
#Locate Size/Sqft details in the <span> tag with the 'housing' class
listing1_bdrmsSqft = listing1.find('span', class_ = 'housing')
listing1_bdrmsSqft

<span class="housing">
                    1br -
                    700ft<sup>2</sup> -
                </span>

In [11]:
#Isolate the number of bedrooms by isolating the first element in the text
#Removed the last 2 characters to produce an int value
listing1_bedrooms = int(listing1_bdrmsSqft.text.split()[0][:-2])
listing1_bedrooms

1

In [12]:
#Isolate the number of bedrooms by isolating the 3rd element in the text
#Removed the last 3 characters to produce an int value
# listing1_Sqft = int(listing1_bdrmsSqft.text.split()[2][:-3])
# listing1_Sqft

In [13]:
#Isolate the neighborhood location by locating the <span> tag with the 'result-hood' class
listing1_neighborhood = listings[0].find('span', class_='result-hood').text
listing1_neighborhood

' (4500 Woodman Ave, Sherman Oaks, CA san fernando valley )'

# Expand data collected by setting up the for loop 

In [28]:
listing_titles= []
listing_hyperlinks = []
listing_neighborhoods = []
listing_datesTimes = []
listing_prices = []
listing_sqFts = []
listing_bdrms = []

pages = np.arange(0, 2, 120)

iterations = 0

response = get(url
            + "s="
            + str(pages))
html = response.text
soup = BeautifulSoup(html, "html.parser")

sleep(randint(1,5))

for listing in listings:

        if listing.find('span', class_ = 'result-hood') is not None: 
          
            try:
                listing_dateTime = listing.find('time', class_= 'result-date')['datetime']   
            except:
                listing_dateTime = None
            listing_datesTimes.append(listing_dateTime)
            
            try:
                listing_title = listing.find('a', class_= 'result-title')
            except: 
                listing_title = None
            listing_titles.append(listing_title)
             
            try:
                listing_hyperlink = listing_title['href']
            except:
                listing_hyperlink = None
            listing_hyperlinks.append(listing_hyperlink)
            
            try:
                listing_neighborhood = listing.find('span', class_= 'result-hood').text
            except:
                listing_neighborhood = None
            listing_neighborhoods.append(listing_neighborhood)
            
            try:
                listing_price = int(listing.a.text.strip().replace("$", "").replace(",", ""))        
            except: 
                listing_price = None 
            listing_prices.append(listing_price)

            

            
            spanHousing = listing.find('span', class_ = 'housing')
            if spanHousing is not None:
                content = spanHousing.Text
            else: 
                content = None
 
                #make bedroom nan
                listing_bdrm = np.nan
                listing_bdrms.append(listing_bdrm)

                #make sqft the first element
                try:
                    listing_sqFt = int(listing.find('span', class_ = 'housing').text.split()[0][:-3])
                except:
                    listing_sqFt = None    
                listing_sqFts.append(listing_sqFt)
                    
            spanHousing = listing.find('span', class_ = 'housing')
            if spanHousing is not None:
                content = spanHousing.Text
            else: 
                content = None
                
                #therefore element 0 will be bedroom count
                try:
                    listing_bdrm = listing.find('span', class_ = 'housing').text.replace("br", "").split()[0]
                except:
                    listing_bdrm = None
                listing_bdrms.append(listing_bdrm)

                #and sqft will be number 3, so set these here and append
                try:                
                    listing_sqFt = int(listing.find('span', class_ = 'housing').text.split()[2][:-3])
                except:
                    listing_sqFt = None    
                listing_sqFts.append(listing_sqFt)
                    
                    
            spanHousing = listing.find('span', class_ = 'housing')
            if spanHousing is not None:
                content = spanHousing.Text
            else: 
                content = None

                #therefore element 0 will be bedroom count
                try:
                    listing_bdrm = listing.find('span', class_ = 'housing').text.replace("br", "").split()[0]
                except:
                    listing_bdrm = None
                listing_bdrms.append(listing_bdrm)

                #and sqft will be number 3, so set these here and append
                listing_sqFt = np.nan
                listing_sqFts.append(listing_sqFt)                    

#             else:
#                 listing_bdrm = np.nan
#                 listing_bdrms.append(listing_bdrm)

#                 listing_sqFt = np.nan
#                 listing_sqFts.append(listing_sqFt)
            
iterations += 1
print("Page " + str(iterations) + " scraped successfully!")
print("\n")
print("Scraped " + str(iterations) + " page(s)")

Page 1 scraped successfully!


Scraped 1 page(s)


In [15]:
LA_Listings_df = pd.DataFrame({'Posted on': listing_datesTimes,
                            'Title': listing_titles,
                            'Link': listing_hyperlinks,
                            'Neighborhood': listing_neighborhoods,
                            'Price' : listing_prices,
                            'Bedrooms' : listing_bdrms,
                            'SqFt' : listing_sqFts
                           })
LA_Listings_df

ValueError: arrays must all be same length

In [ ]:
LA_Listings_df.to_csv("Resources/la_listings_clean.csv")